In [1]:
from marked_words import marked_words
import pandas as pd
from collections import Counter

def pprint(dic):
    full_list = []
    for word in sorted(dic,key=lambda x: x[1],reverse=True):
        #print("%s, %.2f" % (word[0],word[1]))
        full_list.append(word[0])
    return full_list


In [2]:
models=["4o","4v","Gemini","LLaVA","llama","Midjourney"]

In [4]:
count_race_lst = {} 
count_gender_lst = {} 
all_mw={}
for model in models:
    df = pd.read_csv(f'../../result/image_description/{model}//all_data.csv')
    df["text"]=df['text'].apply(lambda x: x.rstrip(":1234567890 "))
    
    mw = {}
    race_set = set(df['race'].unique())
    gender_set = set(df['gender'].unique())
    for race in df['race'].unique():
        outs = pprint(marked_words(df, [race], ['race'],['white']))
        mw[race] = outs
    temps = []
    for race in df['race'].unique():
        temp = pprint(marked_words(df, ['white'], ['race'],[race]))
        temps.extend(temp)
    seen = Counter(temps).most_common()

    temp=[w for w, c in seen if c == 2]
    mw['white']=temp[:10]


    for race in df['gender'].unique():
        outs = pprint(marked_words(df, [race], ['gender'],['male']))
        mw[race] = outs
    temps = []
    for race in df['gender'].unique():
        temp = pprint(marked_words(df, ['male'], ['gender'],[race]))
        temps.extend(temp)

    seen = Counter(temps).most_common()
    mw['male']=[w for w, c in seen if c == 1]
    
    all_mw[model]=mw
    for k,v in mw.items():

        if k in race_set:
            for word in v[:10]:
                if word not in count_race_lst:
                    count_race_lst[word] = set()
                count_race_lst[word].add(model)
        else:
            for word in v[:10]:
                if word not in count_gender_lst:
                    count_gender_lst[word] = set()
                count_gender_lst[word].add(model)
    
                
for model, mw in all_mw.items():
    print(model + ": ")
    for k, v in mw.items():
        print(k + ": ", end="[")
        if k in race_set:
            count_dict = count_race_lst
        else:
            count_dict = count_gender_lst

        for word in v[:10]:
            count = len(count_dict.get(word, set()))
            if count == 6:
                print(r"\colorbox{myred!100}{" + word + "}", end=", ")
            elif count == 5:
                print(r"\colorbox{myred!60}{" + word + "}", end=", ")
            elif count == 4:
                print(r"\colorbox{myred!20}{" + word + "}", end=", ")
            else:
                print(word, end=", ")
        print("]")

4o: 
asian: [\colorbox{myred!60}{desk}, lady, \colorbox{myred!60}{laptop}, \colorbox{myred!20}{statue}, justice, office, dessert, cup, gavel, rescue, ]
white: [his, herbs, wig, pulled, \colorbox{myred!20}{pasta}, barrister, seafood, mushrooms, tied, hair, ]
black: [\colorbox{myred!20}{fire}, tattoos, apron, truck, courtroom, braids, burger, window, sparks, binders, ]
latino: [\colorbox{myred!20}{books}, truck, filled, volumes, bakery, usa, dark, \colorbox{myred!20}{fire}, pot, library, ]
ME: [\colorbox{myred!100}{hijab}, \colorbox{myred!20}{headscarf}, \colorbox{myred!20}{cultural}, two, beige, woman, city, wrapped, evening, court, ]
female: [\colorbox{myred!100}{her}, \colorbox{myred!100}{she}, \colorbox{myred!100}{woman}, hair, \colorbox{myred!100}{hijab}, back, \colorbox{myred!20}{blazer}, \colorbox{myred!60}{female}, blouse, styled, ]
male: [\colorbox{myred!100}{his}, \colorbox{myred!100}{he}, \colorbox{myred!100}{man}, \colorbox{myred!100}{him}, \colorbox{myred!100}{tie}, \colorbo